In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# typical model: Sequential : one input, one output
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
# Funtional API (Single output)
inputs = keras.Input(shape=(28,28))

flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10)

x = flatten(inputs)
x = dense1(x)
outputs = dense2(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='functional_model')
print(model.summary())

Model: "functional_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
# Funtional API (Multiple outputs)
inputs = keras.Input(shape=(28,28))

flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10)
dense2_2 = keras.layers.Dense(1)

x = flatten(inputs)
x = dense1(x)

outputs = dense2(x)
outputs2 = dense2_2(x)
model1 = keras.Model(inputs=inputs, outputs=[outputs, outputs2], name='functional_model')
print(model1.summary())

Model: "functional_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten_2 (Flatten)            (None, 784)          0           ['input_2[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 128)          100480      ['flatten_2[0][0]']              
                                                                                                  
 dense_5 (Dense)                (None, 10)           1290        ['dense_4[0][0]']                
                                                                                   

In [5]:
# Let's convert a sequential model to a functional model
new_model = keras.models.Sequential()
for layer in model.layers:
  new_model.add(layer)
  
# Convert Functional model to a Sequential model
inputs = keras.Input(shape = (28,28))
x = new_model.layers[0](inputs)
for layer in new_model.layers[1:]:
  x = layer(x)
outputs = x

In [6]:
""" Advantages of Functional API:

1. Models with multiple inputs and outputs
2. Shared Layers
3. Extract and reuse nodes in the graph of layers
(Models are callable like layers (put model into sequential))
"""
inputs = model.inputs
outputs = model.outputs

input0 = model.layers[0].input
output0 = model.layers[0].output

In [7]:
print(inputs)
print(outputs)
print(input0)
print(output0)

[<KerasTensor: shape=(None, 28, 28) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_3')>]
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [ ]:
# application of functional api in transfer learning
base_model = keras.applications.VGG16()

x = base_model.layers[-2].output
new_outputs = keras.layers.Dense(1)(x)
new_model = keras.Model(inputs=base_model.inputs, outputs=new_outputs)

#### Let us explore the Functional API in more detail through a real application , 
**We need to predict whether a digit in mnist dataset was written with right hand or left hand**

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# Define Functional Model
inputs = keras.Input(shape=(28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10, activation='softmax', name='category_output')
dense3 = keras.layers.Dense(1, activation='sigmoid', name='leftright_output')

In [3]:
x = flatten(inputs)
x = dense1(x)
outputs1 = dense2(x)
outputs2 = dense3(x)
model = keras.Model(inputs=inputs, outputs=[outputs1, outputs2], name='mnist_model')

model.summary()

Model: "mnist_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten (Flatten)              (None, 784)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 128)          100480      ['flatten[0][0]']                
                                                                                                  
 category_output (Dense)        (None, 10)           1290        ['dense[0][0]']                  
                                                                                        

In [5]:
# loss and optimizer
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

losses = {
    "category_output": loss1,
    "leftright_output": loss2
}

model.compile(loss=losses, optimizer=optim, metrics=metrics)

In [6]:
# create data with 2 labels
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # normalization

# Dummy dataset for left and right digits creation
# 0 = left, 1 = right
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)

for idx, y in enumerate(y_train):
  if y > 5:
    y_leftright[idx] = 1 # each digit greater than 5 written by a right hand

print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])

y = {"category_output": y_train,
     "leftright_output": y_leftright}

11501568/11490434 [==============================] - 0s 0us/step
uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [7]:
# training
model.fit(x_train, y=y, epochs=5, batch_size=64, verbose=2)

Epoch 1/5
938/938 - 3s - loss: 0.5108 - category_output_loss: 0.3139 - leftright_output_loss: 0.1969 - category_output_accuracy: 0.9125 - leftright_output_accuracy: 0.9277 - 3s/epoch - 3ms/step
Epoch 2/5
938/938 - 2s - loss: 0.2425 - category_output_loss: 0.1446 - leftright_output_loss: 0.0980 - category_output_accuracy: 0.9577 - leftright_output_accuracy: 0.9667 - 2s/epoch - 2ms/step
Epoch 3/5
938/938 - 2s - loss: 0.1781 - category_output_loss: 0.1028 - leftright_output_loss: 0.0752 - category_output_accuracy: 0.9704 - leftright_output_accuracy: 0.9747 - 2s/epoch - 2ms/step
Epoch 4/5
938/938 - 2s - loss: 0.1424 - category_output_loss: 0.0799 - leftright_output_loss: 0.0624 - category_output_accuracy: 0.9764 - leftright_output_accuracy: 0.9783 - 2s/epoch - 2ms/step
Epoch 5/5
938/938 - 2s - loss: 0.1171 - category_output_loss: 0.0637 - leftright_output_loss: 0.0534 - category_output_accuracy: 0.9811 - leftright_output_accuracy: 0.9818 - 2s/epoch - 2ms/step


In [8]:
# list with 2 predictions
predictions = model.predict(x_test)
len(predictions)

2

In [9]:
prediction_category = predictions[0]
prediction_lr = predictions[1]

pr_cat = prediction_category[0:20]
prediction_lr = prediction_lr[0:20]

labels_cat = np.argmax(pr_cat, axis=1)
labels_lr = np.array([1 if p>= 0.5 else 0 for p in prediction_lr])

In [10]:
print(y_test[0:20])
print(labels_cat)
print(labels_lr)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0]
